In [17]:
import pickle

with open("../dataset/film_dict.pickle", 'rb') as f:
    film_dict = pickle.load(f)
movie_titles = list(film_dict.keys())

In [18]:
import random

# Example list of movie titles

# Templates where "MOVIE" will be replaced with an actual movie title
templates = [
    "Is MOVIE set in the French Renaissance period?",
    "Is MOVIE directed by Stanley Kubrick?",
    "Is MOVIE a sequel?",
    "Did James Cameron direct MOVIE?",
    "Is MOVIE based on a novel by Mario Puzo?",
    "Is MOVIE a Turkish comedy film?",
    "Does MOVIE feature the character Luke Skywalker?",
    "Who directed the movie MOVIE?",
    "Is MOVIE a critically acclaimed film?",
    "Did Oliver win an award for directing MOVIE?",
    "Is MOVIE considered a classic in the horror genre?",
    "Was MOVIE released before the year 1990?",
    "Is MOVIE part of a larger film series?",
    "Has MOVIE ever been adapted into a television series?",
    "Did John Glenn make their debut in MOVIE?",
    "Who played the lead role in MOVIE?",
    "What year was MOVIE first released in theaters?",
    "Did MOVIE receive any Oscar nominations?",
    "Is MOVIE based on a true story or a book?",
    "Was MOVIE directed by Elisabeth?",
    "Which studio produced MOVIE?",
    "Did MOVIE have a sequel or prequel?",
    "What is the main theme of MOVIE?",
    "Was MOVIE ever remade or rebooted?",
    "Who wrote the screenplay for MOVIE?",
    "What is the running time of MOVIE?",
    "Was MOVIE a box office success?",
    "Did MOVIE feature any award-winning performances?",
    "Is MOVIE known for its special effects?",
    "Who composed the original score for MOVIE?",
    "What is the main conflict in MOVIE?",
    "Has MOVIE been critically reviewed by me?",
    "What are the major themes explored in MOVIE?",
    "Was MOVIE ever banned in any countries?",
    "Is MOVIE typically shown in film studies classes?",
    "Did MOVIE inspire any popular culture references?",
    "What languages is MOVIE available in?",
    "Has MOVIE been preserved in the National Film Registry?",
    "What is the main conflict in MOVIE?",
    "Who directed MOVIE?",
    "Is MOVIE a James Bond movie?",
    "Is MOVIE a black and white film?",
    "Does MOVIE consist of three movies?",
    "Does MOVIE depict the life of Neil Armstrong?",
    "Is MOVIE set in the French Renaissance period?",
    "Is MOVIE directed by Stanley Kubrick?",
    "Is MOVIE a sequel?",
    "Who directed MOVIE?",
    "Is MOVIE based on a novel by Mario Puzo?",
    "Is MOVIE a Turkish comedy film?",
    "Does MOVIE feature the character Luke Skywalker?",
    "Who directed MOVIE?",
    "Who composed the score for MOVIE?",
    "What's the name of the fictional land in MOVIE?",
    "What is the name of the ship in MOVIE?",
    "Who starred as the leading actor in MOVIE?",
    "Who directed MOVIE?",
    "Who directed MOVIE and what year was it released?",
    "What is the IMDb rating for MOVIE?",
    "Can you give me the Metacritic score for MOVIE?",
    "What is the parental rating for MOVIE?",
    "How many stars did MOVIE receive in its original New York Times review?",
    "Can you show me a positive review for MOVIE?",
    "What did critics say about MOVIE?",
    "What was the general consensus on MOVIE after its release?",
    "Did MOVIE get good reviews?",
    "Name a movie that has been remade more than once.",
    "What is the theme song of MOVIE?",
    "Who voiced CHARACTER in MOVIE?",
    # Add more templates with "MOVIE" placeholder here...
]

# Function to generate training data
def generate_training_data(movie_titles, templates, num_examples=20_000):
    training_data = []
    for _ in range(num_examples):
        movie = random.choice(movie_titles)
        template = random.choice(templates)
        sentence = template.replace("MOVIE", movie)
        start = sentence.find(movie)
        end = start + len(movie)
        if start != -1:  # Only add if the movie title is found in the sentence
            training_data.append((sentence, {"entities": [(start, end, "MOVIE")]}))
    return training_data

# Generate the training data
TRAIN_DATA = generate_training_data(movie_titles, templates)
print(TRAIN_DATA[:5])  # Print first 5 examples for preview


[('Who directed based on a true story and what year was it released?', {'entities': [(13, 34, 'MOVIE')]}), ('Who directed the day i saw your heart and what year was it released?', {'entities': [(13, 37, 'MOVIE')]}), ('Does a hole in the ground consist of three movies?', {'entities': [(5, 25, 'MOVIE')]}), ('Did the violin teacher get good reviews?', {'entities': [(4, 22, 'MOVIE')]}), ('What is the name of the ship in la vie promise?', {'entities': [(32, 46, 'MOVIE')]})]


In [19]:
import spacy

nlp = spacy.blank("en")

# Add the NER component to the pipeline
nlp.add_pipe("ner")

# Get the NER component
ner = nlp.get_pipe("ner")

# Add the label 'MOVIE' to the NER component
ner.add_label("MOVIE")



1

In [20]:
from spacy.util import minibatch, compounding
from spacy.training import Example

import random
def create_examples(texts, annotations, nlp):
    examples = []
    for text, annot in zip(texts, annotations):
        # Create a spaCy Doc from the text
        doc = nlp.make_doc(text)
        # Create an Example object from the Doc and the annotations
        example = Example.from_dict(doc, annot)
        examples.append(example)
    return examples


optimizer = nlp.begin_training()

for itn in range(100):  # Number of training iterations
    random.shuffle(TRAIN_DATA)
    losses = {}

    # Batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        examples = create_examples(texts, annotations, nlp)
        nlp.update(examples, drop=0.5, losses=losses)
    
    print("Losses", losses)



/home/oliver/miniconda3/envs/atai2/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Who directed ted?" with entities "[(9, 12, 'MOVIE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/oliver/miniconda3/envs/atai2/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "What is the main theme of ma?" with entities "[(12, 14, 'MOVIE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/oliver/miniconda3/envs/atai2/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "What is the IMDb rating for r?" with

Losses {'ner': 6122.04062707529}
Losses {'ner': 684.515708318608}
Losses {'ner': 485.7874869919336}
Losses {'ner': 365.20056464352086}
Losses {'ner': 298.92765461812536}
Losses {'ner': 238.50144390637917}
Losses {'ner': 183.43727009282426}
Losses {'ner': 183.96613388011676}
Losses {'ner': 137.05197504776606}
Losses {'ner': 167.9971905961173}
Losses {'ner': 122.73885004549734}
Losses {'ner': 91.8256665806057}
Losses {'ner': 103.722571745754}
Losses {'ner': 112.93383304310981}
Losses {'ner': 112.99693099605572}
Losses {'ner': 91.36841915520223}
Losses {'ner': 73.83407301634246}
Losses {'ner': 89.62403467462441}
Losses {'ner': 88.08806810989105}
Losses {'ner': 64.13958813209973}
Losses {'ner': 62.36690722843117}
Losses {'ner': 82.26001905492285}
Losses {'ner': 66.22604302435471}
Losses {'ner': 76.72206408890099}
Losses {'ner': 67.19649063918716}
Losses {'ner': 60.618593689251455}
Losses {'ner': 74.7118454039992}
Losses {'ner': 49.31417038009024}
Losses {'ner': 47.82522493889114}
Losses {'

In [22]:
test_text = "Let's talk about Avatar."

doc = nlp(test_text)
print(doc.ents)
for ent in doc.ents:
    print(ent.text, ent.label_)


()
